In [2]:
# Install necessary libraries
import sys 
import matplotlib.pyplot as plt # plotting and data visualization
import matplotlib.image as mpimg
from imutils import resize, paths # simple image manipulation 
from sklearn.ensemble import RandomForestClassifier # machine learning and analysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from imutils import resize, paths # simple image manipulation
import cv2  # image analysis 

ImportError: DLL load failed: The specified module could not be found.

In [ ]:
# define the training directory with the images 
train_data_dir  = "../small_dataset/"

In [ ]:
# sort the image paths so that they are matched with the correct label
imagePaths = sorted(list(paths.list_images(train_data_dir)))

In [ ]:
# do a bit of data science 
import pandas as pd 
import os

classnames = []
num_files = 0
distributions = {}

# loop through the directories and save the number of files of each type 
for subdir, dirs, files in os.walk(train_data_dir):
    directory = subdir.split("/")
    classname = directory[-1]
    classnames.append(classname)

    for file in files:
        num_files+=1
        #print(os.path.join(subdir, file))
        #print(subdir)
        #print(file)
    
    distributions[classname] = num_files
    num_files = 0
    
                  
distributions.pop("")
print(classnames)
print(len(classnames))
print(distributions)

In [ ]:
# Create a class for constructing our image descriptor. We will pass this descriptor to our classification model.

class RGBHist:
    def __init__(self, bins):
        # specify the number of bins that we will need to build the histogram
        self.bins = bins

    def describe(self, image):
        # compute a 3D histogram for each image using the 3 color spaces (R, G, B)
        # in the image. Normalize the histogram to accomodate multiple image
        # sizes if that is the case.
        histogram = cv2.calcHist([image], [0, 1, 2], None, self.bins, [0, 256, 0, 256, 0, 256])
        cv2.normalize(histogram, histogram)

        # return out 3D histogram as a flattened array
        return histogram.flatten()

In [ ]:
# initialize the image descriptor
desc = RGBHist([8, 8, 8])

In [ ]:
# Function to help construct our training matrix
def create_training_data(trainingImgsPath, trainingLablsPath, descriptor):
    # create an empty list to store the images and labels
    trainingfeatures = []

    # sort the image paths so that they are matched with the correct label
    imagePaths = sorted(list(paths.list_images(trainingImgsPath)))

    # loop through the images and load them to disk
    for image in imagePaths:
        
            img = cv2.imread(image)

            # convert the image from BGR to RGB format (opencv's default is BGR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if descriptor:
                # describe the image and update the list of features
                features = descriptor.describe(img)
                trainingfeatures.append(features)
            else: 
                # if there is no descriptor then return the loaded image only and resize the image to half width/height 
                img = resize(img, width=150)
                trainingfeatures.append(img)

    
    
    # load the training labels and only keep the 1st category
    trainingLabels = pd.read_csv(trainingLablsPath)
    trainingLabels = trainingLabels[["healthy"]].to_numpy().astype("uint8")

    return (trainingfeatures, trainingLabels)

In [ ]:
# change string label(s) to numerial classes
def encode_label(binarizer, label):
    
    """ Parameters
        ----------
        binarizer (OBJECT): label binarizer object
        label (STRING) : classname 
        
        Returns
        -------
        label (INT)
    """
    label = binarizer.transform(label)
    return label


# create a label binarizer object
def create_binarizer(labels):
    
    lb = LabelEncoder()
    lb.fit(list(labels))
    return lb